In [196]:
import pandas as pd
import numpy as np

In [197]:
# download best model
import wandb
api = wandb.Api()
run = api.run("deep_learning_hsa/Abgabe_02/8kpqfasr")
model=run.file("model-best.h5").download(replace=True)

In [198]:
from sharedtask02.Model.model import create_traindataset
from sklearn.preprocessing import StandardScaler
# Import Data
from sklearn.preprocessing import MinMaxScaler
# Read in the csv data using pandas
train = pd.read_csv('../../data/01_train/train.csv', header=None)

sampleTest = pd.read_csv('../../data/01_train/sampleSubmission.csv', header=None)
sampleTest=sampleTest.to_numpy()

test_original = pd.read_csv('../../data/01_train/sampleTest.csv', header=None)
test_original=test_original.to_numpy()
test=test_original.copy()

In [199]:
x_train, y_train, x_test, y_test = create_traindataset(data=train)
scaler = StandardScaler()
#scal Data

scaler.fit_transform(x_train.reshape(-1, 1))

shape = x_train.shape
x_train = scaler.transform(x_train.reshape(-1, 1))
x_train = x_train.reshape(shape)

shape = x_test.shape
x_test = scaler.transform(x_test.reshape(-1, 1))
x_test = x_test.reshape(shape)

shape = test.shape
test = scaler.transform(test.reshape(-1, 1))
test = test.reshape(shape)

print("Data successful preprocessed!")

Data successful preprocessed!


In [200]:
#test if valide
test_reinitalisiert=scaler.inverse_transform(test)

if np.array_equal(test_original,test_reinitalisiert):
    print('Erfolg')
else:
    print('Rundungsfehler')
    for r in range(test_original.shape[0]):
        for c in range(test_original.shape[1]):
            if test_original[r, c] != round(test_reinitalisiert[r, c],10):
                print(r,c,test_original[r,c],test_reinitalisiert[r,c])


Rundungsfehler


In [201]:
import keras

model = keras.models.load_model('model-best.h5')

In [202]:
import tensorflow as tf
test = np.expand_dims(np.array(test), axis=-1)
test = tf.convert_to_tensor(test)
pred_sample = model.predict(x=test)
pred_sample_ausgabe=np.ndarray.tolist(pred_sample)


1/1 [==============================] - 1s 1s/step


In [203]:
from sklearn.metrics import mean_squared_error

#pred_test = model.predict(x=x_train)
print(mean_squared_error(pred_sample, sampleTest))

pred_train = model.predict(x=x_train)
y_train =    y_train.reshape(pred_train.shape)
#pred_train = np.expand_dims(np.array(pred_train), axis=-1)
print(mean_squared_error(pred_train, y_train))

pred_test = model.predict(x=x_test)
y_test =    y_test.reshape(pred_test.shape)
print(mean_squared_error(pred_test, y_test))


10.105548865137234
1555/1555 [==============================] - 6s 3ms/step
11.345508703695149
445/445 [==============================] - 2s 3ms/step
10.396480624465164


In [204]:
pred_test-y_test

array([[-3.75547972, -2.15407133, -5.24843259, ..., -4.16817303,
         1.13177376,  5.45836177],
       [-0.42599487, -4.06473751, -3.86668963, ...,  1.8422852 ,
         6.15279737,  1.9900342 ],
       [-5.32740541, -4.9012558 , -4.05272479, ...,  5.46845641,
         1.43414817, -2.38398161],
       ...,
       [-1.72877817, -0.17999461,  0.22138076, ...,  0.72062019,
         0.83957701,  1.39164057],
       [-0.34218159,  0.13800048,  1.21898321, ...,  0.70720356,
         1.23627021, -2.19131356],
       [-1.14421691,  0.335807  , -0.15318748, ...,  0.69402244,
        -2.71712881, -1.70314169]])

In [205]:
# predict Data

#pred_sampleTest= test.append(pred_sampleTest)
np.savetxt("../../data/01_train/pred_sample.csv",
           pred_sample_ausgabe,
           delimiter =", ",
           fmt ='% s')

print("pred_test.csv successful saved!")

pred_test.csv successful saved!
